**Part 1: Logistic regression: Without prior feature selection**

*   lasso: shrinks coeff to 0 --> already performs feature selection
*   ridge: shrinks coeff towards 0 --> no 'explicit' feature selection, but unimportant variables have a lower coeff
*   note: outcome has multiple classes, so need to use multinomial

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, accuracy_score

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/SPH6004/full_data.csv')
column_names = list(data.columns)
print(column_names)

['gender', 'admission_age', 'race', 'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'sbp_min', 'sbp_max', 'sbp_mean', 'dbp_min', 'dbp_max', 'dbp_mean', 'mbp_min', 'mbp_max', 'mbp_mean', 'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temperature_vital_min', 'temperature_vital_max', 'temperature_vital_mean', 'spo2_min', 'spo2_max', 'spo2_mean', 'glucose_vital_min', 'glucose_vital_max', 'glucose_vital_mean', 'hematocrit_lab_min', 'hematocrit_lab_max', 'hemoglobin_lab_min', 'hemoglobin_lab_max', 'platelets_min', 'platelets_max', 'wbc_min', 'wbc_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_lab_min', 'bicarbonate_lab_max', 'bun_min', 'bun_max', 'calcium_lab_min', 'calcium_lab_max', 'chloride_lab_min', 'chloride_lab_max', 'creatinine_min', 'creatinine_max', 'glucose_lab_min', 'glucose_lab_max', 'sodium_lab_min', 'sodium_lab_max', 'potassium_lab_min', 'potassium_lab_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'ptt_min', 'ptt_max', 'gcs_min', 'gcs_motor', 'gcs_verbal', 'g


*   Gender and race are nominal, so theres no meaningful ordering --> need to use one-hot encoding for these
*   Rest of the variables are numeric, and have already been scaled, NAs have been imputed w kNN with aki removed to prevent data leakage (refer to r script)

In [ ]:
nominal_cols = ['gender', 'race']

def one_hot_encoding(df_orig, nominal_col):
    dummies = pd.get_dummies(df_orig[[nominal_col]])
    res = pd.concat([df_orig, dummies], axis=1)
    res = res.drop([nominal_col], axis=1)
    return(res)

data = one_hot_encoding(data, 'gender')
data = one_hot_encoding(data, 'race')

print(list(data.columns))

['admission_age', 'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'sbp_min', 'sbp_max', 'sbp_mean', 'dbp_min', 'dbp_max', 'dbp_mean', 'mbp_min', 'mbp_max', 'mbp_mean', 'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temperature_vital_min', 'temperature_vital_max', 'temperature_vital_mean', 'spo2_min', 'spo2_max', 'spo2_mean', 'glucose_vital_min', 'glucose_vital_max', 'glucose_vital_mean', 'hematocrit_lab_min', 'hematocrit_lab_max', 'hemoglobin_lab_min', 'hemoglobin_lab_max', 'platelets_min', 'platelets_max', 'wbc_min', 'wbc_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_lab_min', 'bicarbonate_lab_max', 'bun_min', 'bun_max', 'calcium_lab_min', 'calcium_lab_max', 'chloride_lab_min', 'chloride_lab_max', 'creatinine_min', 'creatinine_max', 'glucose_lab_min', 'glucose_lab_max', 'sodium_lab_min', 'sodium_lab_max', 'potassium_lab_min', 'potassium_lab_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'ptt_min', 'ptt_max', 'gcs_min', 'gcs_motor', 'gcs_verbal', 'gcs_eyes', 'gcs_una

In [ ]:
X = data.drop('aki', axis=1)
Y = data['aki']
Y.value_counts()

,count
aki,
0,16347
2,16179
1,9735
3,7871


Data is now ready for regression
1.   Need to split the data into training and testing splits by aki (which is Y)
2.   Since we are trying both l1 and l2 regularisation, and we need multiclass classification --> need to use the saga solver
3. Smaller values of C = stronger regularisation (inverse of regularisation strength) [refer here for scikit-learn log regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
4. Used 5-fold cross validation for training set --> more robust estimates
5. imo data is relatively balanced, so i used accuracy as the score instead of f1

In [ ]:
#split into final training and testing set, save to csv so it can be used directly later
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=123, stratify=Y
)

X_train.to_csv('/content/drive/MyDrive/SPH6004/X_train.csv')
X_test.to_csv('/content/drive/MyDrive/SPH6004/X_test.csv')
y_train.to_csv('/content/drive/MyDrive/SPH6004/y_train.csv')
y_test.to_csv('/content/drive/MyDrive/SPH6004/y_test.csv')

In [ ]:
param_grid_L1 = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1']
}

param_grid_L2 = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2']
}

lr = LogisticRegression(
    multi_class='multinomial',
    solver='saga',
    max_iter=5000,
    random_state=123
)

#stratified splitting by 5 folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

grid_search = GridSearchCV(
    lr,
    param_grid=param_grid_L1,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1
)

grid_search_2 = GridSearchCV(
    lr,
    param_grid=param_grid_L2,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train)
results_df = pd.DataFrame(grid_search.cv_results_)
print("Paramter fits for L1 regularisation:")
print(results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Paramter fits for L1 regularisation:
                         params  mean_test_score  std_test_score  \
0  {'C': 0.01, 'penalty': 'l1'}         0.479466        0.004474   
1   {'C': 0.1, 'penalty': 'l1'}         0.483556        0.005030   
2     {'C': 1, 'penalty': 'l1'}         0.484304        0.005639   
3    {'C': 10, 'penalty': 'l1'}         0.484204        0.005528   
4   {'C': 100, 'penalty': 'l1'}         0.484179        0.005477   

   rank_test_score  
0                5  
1                4  
2                1  
3                2  
4                3  


In [ ]:
grid_search_2.fit(X_train, y_train)
results_df = pd.DataFrame(grid_search_2.cv_results_)
print("Paramter fits for L2 regularisation:")
print(results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Paramter fits for L2 regularisation:
                         params  mean_test_score  std_test_score  \
0  {'C': 0.01, 'penalty': 'l2'}         0.483606        0.004743   
1   {'C': 0.1, 'penalty': 'l2'}         0.484279        0.005496   
2     {'C': 1, 'penalty': 'l2'}         0.484179        0.005356   
3    {'C': 10, 'penalty': 'l2'}         0.484129        0.005429   
4   {'C': 100, 'penalty': 'l2'}         0.484204        0.005517   

   rank_test_score  
0                5  
1                1  
2                3  
3                4  
4                2  


In [ ]:
best_model_L1 = grid_search.best_estimator_
y_pred_L1 = best_model_L1.predict(X_test)
print("Classification Report for L1 Regularisation:")
print(classification_report(y_test, y_pred_L1))
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_L1):.5f}")

Classification Report for L1 Regularisation:
              precision    recall  f1-score   support

           0       0.52      0.68      0.59      3270
           1       0.33      0.02      0.04      1947
           2       0.45      0.64      0.53      3236
           3       0.57      0.34      0.42      1574

    accuracy                           0.49     10027
   macro avg       0.47      0.42      0.40     10027
weighted avg       0.47      0.49      0.44     10027

Test Accuracy: 0.48868


In [ ]:
best_model_L2 = grid_search_2.best_estimator_
y_pred_L2 = best_model_L2.predict(X_test)
print("Classification Report for L2 Regularisation:")
print(classification_report(y_test, y_pred_L2))
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_L2):.5f}")

Classification Report for L2 Regularisation:
              precision    recall  f1-score   support

           0       0.52      0.68      0.59      3270
           1       0.33      0.02      0.04      1947
           2       0.45      0.64      0.53      3236
           3       0.56      0.34      0.42      1574

    accuracy                           0.49     10027
   macro avg       0.47      0.42      0.40     10027
weighted avg       0.47      0.49      0.44     10027

Test Accuracy: 0.48848


LightGBM to try a tree-based approach with gradient boosting
1. Pros: fast, so it will be easier to do grid search without waiting for too long
2. has an in-built feature selection process, so i can tune the hyperparameters at the same time as performing feature selection

In [14]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

In [6]:
X_test = pd.read_csv('/content/drive/MyDrive/SPH6004/X_test.csv',index_col=0)
X_train = pd.read_csv('/content/drive/MyDrive/SPH6004/X_train.csv',index_col=0)
y_test = pd.read_csv('/content/drive/MyDrive/SPH6004/y_test.csv',index_col=0)
y_train = pd.read_csv('/content/drive/MyDrive/SPH6004/y_train.csv',index_col=0)
column_names = list(X_test)
print(column_names)

['admission_age', 'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'sbp_min', 'sbp_max', 'sbp_mean', 'dbp_min', 'dbp_max', 'dbp_mean', 'mbp_min', 'mbp_max', 'mbp_mean', 'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temperature_vital_min', 'temperature_vital_max', 'temperature_vital_mean', 'spo2_min', 'spo2_max', 'spo2_mean', 'glucose_vital_min', 'glucose_vital_max', 'glucose_vital_mean', 'hematocrit_lab_min', 'hematocrit_lab_max', 'hemoglobin_lab_min', 'hemoglobin_lab_max', 'platelets_min', 'platelets_max', 'wbc_min', 'wbc_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_lab_min', 'bicarbonate_lab_max', 'bun_min', 'bun_max', 'calcium_lab_min', 'calcium_lab_max', 'chloride_lab_min', 'chloride_lab_max', 'creatinine_min', 'creatinine_max', 'glucose_lab_min', 'glucose_lab_max', 'sodium_lab_min', 'sodium_lab_max', 'potassium_lab_min', 'potassium_lab_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'ptt_min', 'ptt_max', 'gcs_min', 'gcs_motor', 'gcs_verbal', 'gcs_eyes', 'gcs_una

In [7]:
#I wanted to select features above a specific importance threshold, then use those features to train the lgbm
#but i changed my mind lol so pls ignore the feature selection part of the pipeline i was too lazy to rewrite without a pipeline
lgbm_pipeline = Pipeline([
    #('feature_selection', SelectFromModel(lgb.LGBMClassifier(objective='multiclass',random_state=123))),
     ('classifier', lgb.LGBMClassifier(objective='multiclass',random_state=123))
])

param_grid = {
    #'feature_selection__threshold': ['median', 0.01, 0.05],
    'classifier__max_depth': [3, 5, 7, -1], #-1 is no limit on the max depth
    'classifier__subsample' :[1.0, 0.75, 0.5], #this is the proportion of the data that the lgbm sees for each boosting iteration
    'classifier__learning_rate': [1.0, 0.1, 0.01],
    'classifier__n_estimators': [50, 75, 100]
}


In [9]:
grid_search_lgbm = GridSearchCV(lgbm_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lgbm.fit(X_train, y_train.values.ravel())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10101
[LightGBM] [Info] Number of data points in the train set: 40105, number of used features: 72
[LightGBM] [Info] Start training from score -1.120646
[LightGBM] [Info] Start training from score -1.638917
[LightGBM] [Info] Start training from score -1.130946
[LightGBM] [Info] Start training from score -1.851428
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('classifier',
                                        LGBMClassifier(objective='multiclass',
                                                       random_state=123))]),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [1.0, 0.1, 0.01],
                         'classifier__max_depth': [3, 5, 7, -1],
                         'classifier__n_estimators': [50, 75, 100],
                         'classifier__subsample': [1.0, 0.75, 0.5]},
             scoring='accuracy')

In [10]:
results_df = pd.DataFrame(grid_search_lgbm.cv_results_)
print("Paramter fits for LGBM:")
print(results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']])

Paramter fits for LGBM:
                                                params  mean_test_score  \
0    {'classifier__learning_rate': 1.0, 'classifier...         0.468196   
1    {'classifier__learning_rate': 1.0, 'classifier...         0.468196   
2    {'classifier__learning_rate': 1.0, 'classifier...         0.468196   
3    {'classifier__learning_rate': 1.0, 'classifier...         0.464979   
4    {'classifier__learning_rate': 1.0, 'classifier...         0.464979   
..                                                 ...              ...   
103  {'classifier__learning_rate': 0.01, 'classifie...         0.479865   
104  {'classifier__learning_rate': 0.01, 'classifie...         0.479865   
105  {'classifier__learning_rate': 0.01, 'classifie...         0.484304   
106  {'classifier__learning_rate': 0.01, 'classifie...         0.484304   
107  {'classifier__learning_rate': 0.01, 'classifie...         0.484304   

     std_test_score  rank_test_score  
0          0.004141               67

In [11]:
# Sort by mean_test_score in descending order bc higher scores are better
top_results = results_df.sort_values(by="mean_test_score", ascending=False).head(5)

print("Top 5 Parameter Fits for LGBM:")
print(top_results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']])

Top 5 Parameter Fits for LGBM:
                                               params  mean_test_score  \
48  {'classifier__learning_rate': 0.1, 'classifier...         0.497644   
49  {'classifier__learning_rate': 0.1, 'classifier...         0.497644   
50  {'classifier__learning_rate': 0.1, 'classifier...         0.497644   
71  {'classifier__learning_rate': 0.1, 'classifier...         0.497544   
70  {'classifier__learning_rate': 0.1, 'classifier...         0.497544   

    std_test_score  rank_test_score  
48        0.002826                1  
49        0.002826                1  
50        0.002826                1  
71        0.003442                4  
70        0.003442                4  


In [12]:
#3 out of 5 have the same test score wHAT are the paramters??
best_rank_results = results_df[results_df['rank_test_score'] == 1]
best_rank_params = best_rank_results['params'].tolist()

#print best parameter sets
for i in best_rank_params:
  print(i)

{'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 75, 'classifier__subsample': 1.0}
{'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 75, 'classifier__subsample': 0.75}
{'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 75, 'classifier__subsample': 0.5}


In [15]:
#Interestingly, all the best fitted parameter sets have the same learning rate, max_depth, and n_estimators, though their classifier subsample varies
#subsample is not as important it seems, interestinggggg

best_model_lgbm = grid_search_lgbm.best_estimator_
y_pred_lgbm = best_model_lgbm.predict(X_test)
print("Classification Report for lgbm:")
print(classification_report(y_test, y_pred_lgbm))
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_lgbm):.5f}")

Classification Report for lgbm:
              precision    recall  f1-score   support

           0       0.55      0.67      0.61      3270
           1       0.30      0.02      0.04      1947
           2       0.45      0.66      0.54      3236
           3       0.57      0.42      0.48      1574

    accuracy                           0.50     10027
   macro avg       0.47      0.44      0.42     10027
weighted avg       0.47      0.50      0.45     10027

Test Accuracy: 0.50194


In [16]:
#get the top features and re-train the model with the best performing model chosen to see if performance improves
best_model_lgbm = grid_search_lgbm.best_estimator_
chosen_params =grid_search_lgbm.best_params_
selector = SelectFromModel(best_model_lgbm.named_steps['classifier'], threshold="mean", max_features=None)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [17]:
selected_features = selector.get_support(indices=True)
print("Selected features indices:", selected_features)

Selected features indices: [ 0  1  2  4  5  6  7  9 10 11 15 16 17 19 21 22 23 25 26 27 29 30 31 32
 37 38 39 42 43 44 46 54 55 56 59 62]


In [18]:
lgbm_pipeline.set_params(**chosen_params)
lgbm_pipeline.fit(X_train_selected, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011915 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7164
[LightGBM] [Info] Number of data points in the train set: 40105, number of used features: 36
[LightGBM] [Info] Start training from score -1.120646
[LightGBM] [Info] Start training from score -1.638917
[LightGBM] [Info] Start training from score -1.130946
[LightGBM] [Info] Start training from score -1.851428
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

Pipeline(steps=[('classifier',
                 LGBMClassifier(max_depth=5, n_estimators=75,
                                objective='multiclass', random_state=123))])

In [19]:
y_pred_lgbm_new = lgbm_pipeline.predict(X_test_selected)
print("Classification Report for best perfoming lgbm with only important features:")
print(classification_report(y_test, y_pred_lgbm_new))
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_lgbm_new):.5f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Classification Report for best perfoming lgbm with only important features:
              precision    recall  f1-score   support

           0       0.55      0.67      0.60      3270
           1       0.31      0.02      0.04      1947
           2       0.45      0.66      0.53      3236
           3       0.56      0.41      0.48      1574

    accuracy                           0.50     10027
   macro avg       0.47      0.44      0.41     10027
weighted avg       0.47      0.50      0.45     10027

Test Accuracy: 0.50005


Okay... seems like theres not much difference... however!
1.   multinomial logistic regression/lbgm means we lose the "orderedness" of the aki_stage variable when we train --> might need to try another method that accounts for this
2.   accuracy also only tells me the proportion of labels that were classified correctly, so all errors are treated the same --> but this might not be ideal? Because misclassifying a 3 as a 0 should be "penalised" more than misclassifying a 3 as a 2, but this is also related to the use of



In [ ]:
!python --version

Python 3.11.11
